In [5]:
from bs4 import BeautifulSoup
import requests

# HTML From File
with open("index.html", "r") as f:
	doc = BeautifulSoup(f, "html.parser")

tags = doc.find_all("p")[0]

print(tags.find_all("b"))

# HTML From Website
url = "https://www.newegg.ca/gigabyte-geforce-rtx-3080-ti-gv-n308tgaming-oc-12gd/p/N82E16814932436?Description=3080&cm_re=3080-_-14-932-436-_-Product"

result = requests.get(url)
doc = BeautifulSoup(result.text, "html.parser")

prices = doc.find_all(text="$")
parent = prices[0].parent
strong = parent.find("strong")
print(strong.string)

[<b color="red">This is a new paragraph!</b>, <b><i>This is a new sentence without a paragraph break, in bold italics.</i></b>]
1,449


In [6]:
from bs4 import BeautifulSoup
import re

with open("index.html", "r") as f:
	doc = BeautifulSoup(f, "html.parser")

tags = doc.find_all("input", type="text")
for tag in tags:
	tag['placeholder'] = "I changed you!"

with open("changed.html", "w") as file:
	file.write(str(doc))

In [2]:
from bs4 import BeautifulSoup
import requests

url = "https://coinmarketcap.com/"
result = requests.get(url).text
doc = BeautifulSoup(result, "html.parser")

tbody = doc.tbody
trs = tbody.contents

prices = {}

for tr in trs[:10]:
	name, price = tr.contents[2:4]
	fixed_name = name.p.string
	fixed_price = price.a.string

	prices[fixed_name] = fixed_price

print(prices)

{'Bitcoin': '$23,834.77', 'Ethereum': '$1,883.38', 'Tether': '$1.00', 'USD Coin': '$1', 'BNB': '$321.00', 'XRP': '$0.3727', 'Cardano': '$0.5285', 'Binance USD': '$0.9999', 'Solana': '$42.85', 'Polkadot': '$9.29'}


In [1]:
from bs4 import BeautifulSoup
import requests
import re

search_term = input("What product do you want to search for? ")

url = f"https://www.newegg.ca/p/pl?d={search_term}&N=4131"
page = requests.get(url).text
doc = BeautifulSoup(page, "html.parser")

page_text = doc.find(class_="list-tool-pagination-text").strong
pages = int(str(page_text).split("/")[-2].split(">")[-1][:-1])

items_found = {}

for page in range(1, pages + 1):
	url = f"https://www.newegg.ca/p/pl?d={search_term}&N=4131&page={page}"
	page = requests.get(url).text
	doc = BeautifulSoup(page, "html.parser")

	div = doc.find(class_="item-cells-wrap border-cells items-grid-view four-cells expulsion-one-cell")
	items = div.find_all(text=re.compile(search_term))

	for item in items:
		parent = item.parent
		if parent.name != "a":
			continue

		link = parent['href']
		next_parent = item.find_parent(class_="item-container")
		try:
			price = next_parent.find(class_="price-current").find("strong").string
			items_found[item] = {"price": int(price.replace(",", "")), "link": link}
		except:
			pass

sorted_items = sorted(items_found.items(), key=lambda x: x[1]['price'])

for item in sorted_items:
	print(item[0])
	print(f"${item[1]['price']}")
	print(item[1]['link'])
	print("-------------------------------")

What product do you want to search for? BTC
BITEO B75 Mining Motherboard with CPU LGA 1155 8 USB 3.0 to PCIE 16X Support 1660/580 for Mining BTC/ETH/ZEC etc
$119
https://www.newegg.ca/p/1JW-00P9-00030?Description=BTC&cm_re=BTC-_-9SIBA31HF08795-_-Product
-------------------------------
B250 BTC Mining Machine Motherboard 12 PCI-E16X Graph Card SODIMM LGA 1151 DDR4 SATA3.0 Support VGA DVI for Miner
$139
https://www.newegg.ca/p/1JW-00NG-00004?Description=BTC&cm_re=BTC-_-9SIB602HPA7873-_-Product
-------------------------------
BITEO B250C Mining Motherboard LGA 1151 12 USB 3.0 to PCIE X16 Support 1660/2080/3050/3060 DDR4 mSATA for Mining BTC/ETH/ZEC etc
$139
https://www.newegg.ca/p/1JW-00P9-00043?Description=BTC&cm_re=BTC-_-9SIBA31HFG0960-_-Product
-------------------------------
BITEO B250 Mining Motherboard LGA 1151 12 PCIE X1 to PCIE X16 Support 1660/2080/3050/3060 DDR4 mSATA USB 3.0 VGA for Mining BTC/ETH/ZEC etc
$139
https://www.newegg.ca/p/1JW-00P9-00038?Description=BTC&cm_re=BTC-_-9